In [ ]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw

In [ ]:
with np.load('data/octa_sphere_5.npz') as npl:
    v_s, f_s = npl['v'], npl['f']

In [ ]:
v, f = igl.read_triangle_mesh('data/hand.off')
segments_recording = np.zeros(len(v), dtype=int)

In [ ]:
tb = iw.ToggleButtons(description='Segment Choice', options=[1,'+'])
@mp.interact(x=tb)
def s_but(x):
    if x == '+':
        tb.options = list(range(1,len(tb.options)+1)) + ['+']
        tb.value = len(tb.options) - 1
    else:
        tb.current = x

button = iw.Button(description="Paint!")
clear_button = iw.Button(description="Clear!")

# Set Callback
def paint_clicked(b):
    slicer = np.where(np.linalg.norm(v - sf.coord[1:],axis=1) < sf.coord[0])[0]
    segments_recording[slicer] = tb.current
    paint_ui.update_object(oid=0, colors=segments_recording)
def clear_clicked(b):
    segments_recording[segments_recording == tb.current] = 0
    paint_ui.update_object(oid=0, colors=segments_recording)
button.on_click(paint_clicked)
clear_button.on_click(clear_clicked)

# Display Buttons
display(iw.HBox([button, clear_button]))

# Meshplot
paint_ui = mp.plot(v,f,c=segments_recording)
paint_ui.add_mesh(v_s*10, f_s,flat=False,c=np.array([1,0,0]))
def sf(radius,x,y,z):
    paint_ui.update_object(oid = 1, vertices = v_s*radius + np.array([x,y,z]))
    sf.coord = [radius,x,y,z]
mp.interact(sf, 
            radius = iw.FloatSlider(min=1, max=10, value=10),
            x = iw.FloatSlider(min=-30, max=30, value=0),
            y = iw.FloatSlider(min=-30, max=30, value=0),
            z = iw.FloatSlider(min=-30, max=30, value=0))

In [ ]:
np.save('hand_label.npy', segments_recording)